In [ ]:
from data import LipsyncDataset
import torch
from torch import nn
from data import AudioMFCC, CustomMFCC
import matplotlib.pyplot as plt
import torchaudio
import numpy as np
from torch import Tensor
from torchaudio.transforms import AmplitudeToDB, MelSpectrogram
import torchaudio.functional as F

In [ ]:
dataset = LipsyncDataset('../data/lipsync.parquet')

In [ ]:
#audio_raw = dataset[0]['audio']
waveform, sr = torchaudio.load("hello_16000.wav")
rwaveform = waveform #torchaudio.functional.resample(waveform, sr, 16000)
print(waveform.shape)

audio_raw = rwaveform.reshape([-1])
audio_mfcc = AudioMFCC()({ 'audio': audio_raw, 'visemes': [] })['audio'][:26, :]

In [ ]:
audio_raw.shape, audio_mfcc.shape, (audio_raw.shape[0] / 16000 - 25e-3)/ (10e-3)

In [ ]:
def plot_spectrogram(specgram, title=None, xlabel="time", ylabel="freq"):
    _, ax = plt.subplots(1, 1)
    ax.set_ylabel(ylabel)
    ax.set_xlabel(xlabel)
    ax.imshow(specgram, origin="lower", aspect="auto", interpolation="nearest")

In [ ]:
def mfcc(samples, audio_rate=16000, num_mels=13, window_time=25e-3, hop_time=10e-3):
    window_length = round(window_time * audio_rate)
    hop_length = round(hop_time * audio_rate)
    melkwargs = {
        "n_fft": window_length,
        "win_length": window_length,
        "hop_length": hop_length,
        "n_mels": num_mels,
        "center": False,
    }
    custommfcc = CustomMFCC(sample_rate=audio_rate, n_mfcc=num_mels, melkwargs=melkwargs)
    ms = custommfcc.MelSpectrogram.mel_scale
    a = custommfcc(samples)
    d = np.array([0.5, 0.5, -0.5, -0.5])
    x = a.numpy()
    delta_a = np.apply_along_axis(np.convolve, axis=1, arr=x, v=d, mode='same')
    # Stack everything
    return torch.cat((a, torch.tensor(delta_a))).to(torch.float)
    #return a.to(torch.float)


In [ ]:
x = np.array([[1, 2, 3, 4], [3, 2, 2, 1]])
d = np.array([0, 0, -1, -1])
delta = np.apply_along_axis(np.convolve, axis=1, arr=x, v=d, mode='same')
delta

In [ ]:
plot_spectrogram(mfcc(audio_raw))

In [ ]:
plot_spectrogram(audio_mfcc)

In [ ]:
(mfcc(audio_raw) - audio_mfcc).sum()

In [ ]:
import torch
torch.hann_window(5)

In [ ]:
def mfcc2(samples, audio_rate=16000, num_mels=13, window_time=25e-3, hop_time=10e-3):
    window_length = round(window_time * audio_rate)
    fft_length = window_length // 2 + 1
    hop_length = round(hop_time * audio_rate)
    n = len(range(0, samples.shape[0] - window_length, hop_length))
    a = torch.zeros(num_mels, n)
    melbank = F.melscale_fbanks(fft_length, 0.0, audio_rate / 2.0, num_mels, audio_rate, None, 'htk')
    torch.save({ 'test': melbank }, 'melbank.pt')
    hann = torch.hann_window(window_length, periodic=True)

    def ms(x, debug=False):
        # Window with hann window
        #print(f"x = {x.shape} {x}")
        wx = x * hann
        # Do FFT, then do |fft|^2 to get power
        fx = torch.fft.fft(wx).abs().pow(2.0)[:201]
        if debug:
            print(f"pwr = {fx}");
        # fx is shape [201]
        # melbank is shape [201, 13]
        mx = torch.matmul(fx, melbank)
        # mx is [13]
        return mx.unsqueeze(1)
    
    def adb(x):
        return torch.log10(torch.clamp(x, min=1e-10)) * 10.0

    for i in range(n):
        offset = i * hop_length
        slc = torch.zeros(window_length)
        slc[:min(window_length, samples.shape[0] - offset)] = samples[offset:offset + window_length]
        msx = ms(slc, False)
        slc_m = adb(msx)
        a[:, i] = slc_m[:, 0]

    return a

#mfcc2(audio_raw)[:, 50], audio_mfcc[:, 50]
mfcc2(audio_raw)[:, 0], audio_mfcc[:, 0]

In [ ]:
audio_mfcc[:, 50]

In [ ]:
y = torch.fft.fft(torch.tensor([1, 2, 3, 4, 5, 6, 7, 8]))
y.abs().pow(2.0)

In [ ]:
fb = F.melscale_fbanks(400, 0.0, 8000.0, 13, 16000, None, 'htk')

In [ ]:
fb

In [ ]:
torch.hann_window(10)

In [ ]:
25e-3*16e3